# Análisis de Escritos según Ámbito 1: Aspectos de Estructura Global - Sistema de Referencias

Este conjunto de llamadas a la API de OpenAI tiene como propósito retomar la evaluación de lo tres componentes de escritura del ámbito 1: **introducción**, **conclusión y cierre** y **progresión textual** de manera automatizada y consistente.  

Cada llamada aplica un **prompt especializado** en la referencia del aspecto a evaluar, lo cual orienta al modelo a analizar aspectos específicos de la estructura del texto, manteniendo un enfoque **interpretativo y pedagógico**, no meramente calificativo. Todo lo anterior se debe al hecho de que no se tiene aún claridad sobre la asignación de puntaje en base a logro para estas categorías (27 de octubre de 2025).

El objetivo es ofrecer **retroalimentación detallada y útil** para el desarrollo de la competencia escrita, en lugar de limitarse a emitir un juicio numérico.

Los prompts fueron diseñados para:
- Mantener **coherencia conceptual y metodológica** entre las distintas etapas del texto.
- Promover una evaluación **flexible pero rigurosa**, respetando los estándares académicos.
- Generar resultados **estructurados en formato JSON**, fácilmente analizables en bases de datos.

A lo largo de este sistema, cada bloque de código:
1. Carga los textos y reactivos desde una fuente externa.  
2. Genera el prompt correspondiente según el componente textual a evaluar.  
3. Envía la solicitud al modelo de Azure OpenAI.  
4. Recibe y guarda la respuesta en formato **JSON limpio**, con comentarios interpretativos, fortalezas, debilidades y sugerencias de mejora.

IMPORTANTE: Para mantener uniformidad entre las evaluaciones, todas las respuestas siguen el siguiente formato:

```json
{
  "evaluacion": {
    "aspecto_evaluado": "Nombre del componente textual (Introducción, Progresión textual, Conclusión y cierre, etc.)",
    "fortalezas": "Aspectos positivos del texto evaluado.",
    "debilidades": "Aspectos que requieren mejora o revisión.",
    "recomendaciones": "Sugerencias concretas para fortalecer la escritura académica."
  }
}


### Introducción

In [4]:
import os
import json
import pandas as pd
from dotenv import load_dotenv
from openai import AzureOpenAI

load_dotenv()
API_KEY = os.getenv("AZURE_OPENAI_KEY")
ENDPOINT = os.getenv("AZURE_OPENAI_ENDPOINT")
DEPLOYMENT_GPT = os.getenv("AZURE_OPENAI_DEPLOYMENT_GPT")
API_VERSION = os.getenv("AZURE_OPENAI_API_VERSION")

client = AzureOpenAI(
    api_key=API_KEY,
    api_version=API_VERSION,
    azure_endpoint=ENDPOINT
)

data_path = os.path.join("..", "data", "escritos_seleccionados_con_reactivo.xlsx")
df = pd.read_excel(data_path)
textos = df["escrito"].dropna().tolist()
reactivos = df["reactivo"].fillna("").tolist()

prompt_introduccion_ref = """
Eres un evaluador y corrector de escritura académica. 
Tu tarea es evaluar la **INTRODUCCIÓN** de un texto para que cumpla con los siguientes criterios académicos exigidos y responda adecuadamente al REACTIVO planteado.

**Tarea:**  
Analiza el texto de introducción académica proporcionado para **identificar y evaluar la presencia de indicios contextuales básicos** que sitúan el tema dentro de su contexto más amplio.  

**Enfoque:**  
En lugar de buscar **menciones explícitas** de elementos tradicionales como *tema, actores, lugar* y *tiempo*, enfócate en **detectar señales, referencias o sugerencias implícitas** que ofrezcan información sobre estos componentes.  
Tu análisis debe mantener un enfoque **flexible e interpretativo**, permitiendo una comprensión matizada de cómo opera el contexto en la escritura académica.

**Preguntas guía:**  
1. ¿Qué **indicios o referencias implícitas** en la introducción indican el **tema principal**?  
2. ¿Existen **menciones de actores**, directas o indirectas, que sugieran **quiénes participan** en la discusión?  
3. ¿Puedes identificar **pistas contextuales** que revelen el **lugar o entorno** relevante para el tema?  
4. ¿Qué **indicadores temporales** están presentes que ayuden a situar el **momento o periodo** de los eventos o discusiones mencionadas?  

**Salida:**  
Proporciona una **evaluación detallada** de tus hallazgos, destacando la **sutileza y complejidad** de los indicios contextuales presentes en el texto.  
Evita juicios rígidos o forzados y busca una **interpretación reflexiva** que refleje la naturaleza del discurso académico.
El formato de la salida debe ser JSON y seguir esta estructura general:
{{
  "evaluacion": {{
    "aspecto_evaluado": "Nombre breve del criterio o componente (por ejemplo: Introducción, Progresión textual, Conclusión y cierre)",
    "fortalezas": "Describe brevemente los elementos que funcionan bien en el texto, destacando aciertos y claridad.",
    "debilidades": "Explica los aspectos que necesitan mejora, señalando de forma concreta los problemas observados.",
    "recomendaciones": "Sugiere mejoras puntuales y realistas para fortalecer la escritura y el cumplimiento de los criterios académicos."
  }}
}}

**Texto a corregir:**
{essay}

**Reactivo del texto:**
{reactive}
"""

resultados = []

for i, (texto, reactivo) in enumerate(zip(textos, reactivos), start=1):    
    prompt_introduccion_referencias = prompt_introduccion_ref.format(essay=texto, reactive=reactivo)

    print(f"Procesando texto {i}/{len(textos)}...")

    messages = [
        {"role": "system", "content": 'Realiza las tareas de corrección que te solicite para un texto entregado. Devuelve la respuesta exclusivamente en formato JSON válido.'},
        {"role": "user", "content": prompt_introduccion_referencias}
    ]

    try:
        response = client.chat.completions.create(
            model=DEPLOYMENT_GPT,
            messages=messages,
            response_format={"type": "json_object"}
        )

        respuesta_limpia = response.choices[0].message.content.strip()
        resultado_json = json.loads(respuesta_limpia)
        resultados.append({
            "id_texto": i,
            "resultado": resultado_json
        })

    except json.JSONDecodeError as e:
        print(f"Error al parsear JSON en texto {i}: {e}")
        print("Respuesta cruda recibida:")
        print(respuesta_limpia)

output_dir = os.path.join("..", "resultados", "07")
os.makedirs(output_dir, exist_ok=True)
output_path = os.path.join(output_dir, "a1_introduccion_ref.json")

with open(output_path, "w", encoding="utf-8") as f:
    json.dump(resultados, f, ensure_ascii=False, indent=4)

print(f"\nTodos los resultados guardados en {output_path}")

Procesando texto 1/20...
Procesando texto 2/20...
Procesando texto 3/20...
Procesando texto 4/20...
Procesando texto 5/20...
Procesando texto 6/20...
Procesando texto 7/20...
Procesando texto 8/20...
Procesando texto 9/20...
Procesando texto 10/20...
Procesando texto 11/20...
Procesando texto 12/20...
Procesando texto 13/20...
Procesando texto 14/20...
Procesando texto 15/20...
Procesando texto 16/20...
Procesando texto 17/20...
Procesando texto 18/20...
Procesando texto 19/20...
Procesando texto 20/20...

Todos los resultados guardados en ..\resultados\07\a1_introduccion_ref.json


### Conclusión y cierre

In [5]:
import os
import json
import pandas as pd
from dotenv import load_dotenv
from openai import AzureOpenAI

load_dotenv()
API_KEY = os.getenv("AZURE_OPENAI_KEY")
ENDPOINT = os.getenv("AZURE_OPENAI_ENDPOINT")
DEPLOYMENT_GPT = os.getenv("AZURE_OPENAI_DEPLOYMENT_GPT")
API_VERSION = os.getenv("AZURE_OPENAI_API_VERSION")

client = AzureOpenAI(
    api_key=API_KEY,
    api_version=API_VERSION,
    azure_endpoint=ENDPOINT
)

data_path = os.path.join("..", "data", "escritos_seleccionados_con_reactivo.xlsx")
df = pd.read_excel(data_path)
textos = df["escrito"].dropna().tolist()
reactivos = df["reactivo"].fillna("").tolist()


prompt_conclusion_cierre_ref = """
Eres un evaluador y corrector de escritura académica.  
Tu tarea es evaluar la **CONCLUSIÓN Y CIERRE** de un texto para verificar si cumple con los criterios académicos exigidos y responde adecuadamente al REACTIVO planteado.

**Tarea:**  
Analiza el texto de conclusión proporcionado para **identificar y evaluar la calidad del cierre argumentativo**, considerando su capacidad para **sintetizar las ideas principales** y **proyectar reflexiones o implicaciones** derivadas del tema tratado.  

**Enfoque:**  
Evita buscar únicamente frases finales o fórmulas de cierre convencionales.  
En cambio, enfócate en **detectar señales implícitas o explícitas** de síntesis y proyección que indiquen una comprensión global del texto.  
Tu análisis debe mantener un enfoque **reflexivo e interpretativo**, reconociendo la sutileza con que el autor puede articular su cierre académico.

**Preguntas guía:**  
1. ¿Qué **indicios o formulaciones** sugieren una **síntesis** de las ideas principales del texto?  
2. ¿Existen **referencias o proyecciones** que indiquen implicaciones, consecuencias o reflexiones futuras sobre el tema?  
3. ¿La conclusión mantiene **coherencia y pertinencia** con el reactivo planteado?  
4. ¿Se distingue un cierre que **trasciende la simple repetición** de la introducción o ideas previas?

**Criterios clave:**  
1. **Síntesis:** reformula la idea principal del texto de forma clara y en nuevas palabras (sin copiar la introducción).  
2. **Proyección:** plantea implicaciones, consecuencias, reflexiones o contextos futuros relevantes para el tema tratado.  

**Salida:**  
Proporciona una **evaluación detallada** de tus hallazgos, explicando en qué medida el texto logra articular una conclusión efectiva en términos de síntesis y proyección.  
Evita juicios rígidos o forzados y busca una **interpretación reflexiva**, reconociendo la complejidad del cierre académico.
El formato de la salida debe ser JSON y seguir esta estructura general:
{{
  "evaluacion": {{
    "aspecto_evaluado": "Nombre breve del criterio o componente (por ejemplo: Introducción, Progresión textual, Conclusión y cierre)",
    "fortalezas": "Describe brevemente los elementos que funcionan bien en el texto, destacando aciertos y claridad.",
    "debilidades": "Explica los aspectos que necesitan mejora, señalando de forma concreta los problemas observados.",
    "recomendaciones": "Sugiere mejoras puntuales y realistas para fortalecer la escritura y el cumplimiento de los criterios académicos."
  }}
}}

**Texto a evaluar:**  
{essay}

**Reactivo del texto:**  
{reactive}
"""

resultados = []

for i, (texto, reactivo) in enumerate(zip(textos, reactivos), start=1):    
    prompt_conclusion_cierre_referencias = prompt_conclusion_cierre_ref.format(essay=texto, reactive=reactivo)

    print(f"Procesando texto {i}/{len(textos)}...")

    messages = [
        {"role": "system", "content": 'Realiza las tareas de corrección que te solicite para un texto entregado'},
        {"role": "user", "content": prompt_conclusion_cierre_referencias}
    ]

    try:
        response = client.chat.completions.create(
            model=DEPLOYMENT_GPT,
            messages=messages,
            # temperature=0, (No es soportado en Azure OpenAI)
            response_format={"type": "json_object"}
        )

        respuesta_limpia = response.choices[0].message.content.strip()
        resultado_json = json.loads(respuesta_limpia)
        resultados.append({
            "id_texto": i,
            "resultado": resultado_json
        })

    except json.JSONDecodeError as e:
        print(f"Error al parsear JSON en texto {i}: {e}")
        print("Respuesta cruda recibida:")
        print(respuesta_limpia)

output_dir = os.path.join("..", "resultados", "07")
os.makedirs(output_dir, exist_ok=True)
output_path = os.path.join(output_dir, "a1_conclusion_cierre_ref.json")

with open(output_path, "w", encoding="utf-8") as f:
    json.dump(resultados, f, ensure_ascii=False, indent=4)

print(f"\nTodos los resultados guardados en {output_path}")

Procesando texto 1/20...
Procesando texto 2/20...
Procesando texto 3/20...
Procesando texto 4/20...
Procesando texto 5/20...
Procesando texto 6/20...
Procesando texto 7/20...
Procesando texto 8/20...
Procesando texto 9/20...
Procesando texto 10/20...
Procesando texto 11/20...
Procesando texto 12/20...
Procesando texto 13/20...
Procesando texto 14/20...
Procesando texto 15/20...
Procesando texto 16/20...
Procesando texto 17/20...
Procesando texto 18/20...
Procesando texto 19/20...
Procesando texto 20/20...

Todos los resultados guardados en ..\resultados\07\a1_conclusion_cierre_ref.json


### Progresión textual

In [6]:
import os
import json
import pandas as pd
from dotenv import load_dotenv
from openai import AzureOpenAI

load_dotenv()
API_KEY = os.getenv("AZURE_OPENAI_KEY")
ENDPOINT = os.getenv("AZURE_OPENAI_ENDPOINT")
DEPLOYMENT_GPT = os.getenv("AZURE_OPENAI_DEPLOYMENT_GPT")
API_VERSION = os.getenv("AZURE_OPENAI_API_VERSION")

client = AzureOpenAI(
    api_key=API_KEY,
    api_version=API_VERSION,
    azure_endpoint=ENDPOINT
)

data_path = os.path.join("..", "data", "escritos_seleccionados_con_reactivo.xlsx")
df = pd.read_excel(data_path)
textos = df["escrito"].dropna().tolist()
reactivos = df["reactivo"].fillna("").tolist()

prompt_progresion_textual_ref = """
Eres un evaluador y corrector de escritura académica.  
Tu tarea es evaluar la **PROGRESIÓN TEXTUAL** de un texto para verificar si cumple con los criterios académicos exigidos y responde adecuadamente al REACTIVO planteado.

**Tarea:**  
Analiza el texto completo para **identificar la claridad, coherencia y continuidad temática** entre los párrafos.  
Tu objetivo es determinar si el desarrollo del texto avanza de manera ordenada y lógica, permitiendo una lectura fluida y estructurada.

**Enfoque:**  
No busques únicamente la presencia formal de conectores o frases de enlace.  
En cambio, enfócate en **detectar la manera en que las ideas evolucionan, se relacionan y se distinguen entre sí**.  
Evalúa si cada párrafo aporta algo nuevo al argumento y si existe un hilo conductor entre las partes del texto.  
Mantén un enfoque **analítico e interpretativo**, atendiendo a la coherencia global más que a la mera forma.

**Preguntas guía:**  
1. ¿Cada párrafo desarrolla un **subtema claramente distinto**, contribuyendo al avance del argumento general?  
2. ¿Hay **ausencia de repeticiones** o redundancias temáticas entre los párrafos?  
3. ¿Se observan **conectores lógicos o transiciones** que articulen adecuadamente las ideas entre párrafos?  
4. ¿El texto presenta una **progresión fluida**, en la que las ideas se enlazan de manera natural y coherente?

**Criterios clave:**  
1. **Distinción de subtemas:** cada párrafo aborda una idea o aspecto diferente, sin mezclarse con los anteriores.  
2. **Evita repetición:** no hay reiteraciones innecesarias de ideas ya desarrolladas en párrafos previos.  
3. **Conectores lógicos:** se utilizan expresiones de enlace que aseguran continuidad entre párrafos (por ejemplo: además, por otra parte, en consecuencia, sin embargo, etc.).  

**Salida:**  
Proporciona una **evaluación detallada** de tus hallazgos, explicando en qué medida el texto logra mantener una progresión textual clara, coherente y lógica.  
Evita juicios rígidos o superficiales y busca una **interpretación reflexiva**, reconociendo tanto las fortalezas como las debilidades del desarrollo argumentativo.
El formato de la salida debe ser JSON y seguir esta estructura general:
{{
  "evaluacion": {{
    "aspecto_evaluado": "Nombre breve del criterio o componente (por ejemplo: Introducción, Progresión textual, Conclusión y cierre)",
    "fortalezas": "Describe brevemente los elementos que funcionan bien en el texto, destacando aciertos y claridad.",
    "debilidades": "Explica los aspectos que necesitan mejora, señalando de forma concreta los problemas observados.",
    "recomendaciones": "Sugiere mejoras puntuales y realistas para fortalecer la escritura y el cumplimiento de los criterios académicos."
  }}
}}

**Texto a evaluar:**  
{essay}

**Reactivo del texto:**  
{reactive}
"""

resultados = []

for i, (texto, reactivo) in enumerate(zip(textos, reactivos), start=1):    
    prompt_progresion_textual_referencias = prompt_progresion_textual_ref.format(essay=texto, reactive=reactivo)

    print(f"Procesando texto {i}/{len(textos)}...")

    messages = [
        {"role": "system", "content": 'Realiza las tareas de corrección que te solicite para un texto entregado'},
        {"role": "user", "content": prompt_progresion_textual_referencias}
    ]

    try:
        response = client.chat.completions.create(
            model=DEPLOYMENT_GPT,
            messages=messages,
            # temperature=0, (No es soportado en Azure OpenAI)
            response_format={"type": "json_object"}
        )

        respuesta_limpia = response.choices[0].message.content.strip()
        resultado_json = json.loads(respuesta_limpia)
        resultados.append({
            "id_texto": i,
            "resultado": resultado_json
        })

    except json.JSONDecodeError as e:
        print(f"Error al parsear JSON en texto {i}: {e}")
        print("Respuesta cruda recibida:")
        print(respuesta_limpia)

output_dir = os.path.join("..", "resultados", "07")
os.makedirs(output_dir, exist_ok=True)
output_path = os.path.join(output_dir, "a1_progresion_textual_ref.json")

with open(output_path, "w", encoding="utf-8") as f:
    json.dump(resultados, f, ensure_ascii=False, indent=4)

print(f"\nTodos los resultados guardados en {output_path}")

Procesando texto 1/20...
Procesando texto 2/20...
Procesando texto 3/20...
Procesando texto 4/20...
Procesando texto 5/20...
Procesando texto 6/20...
Procesando texto 7/20...
Procesando texto 8/20...
Procesando texto 9/20...
Procesando texto 10/20...
Procesando texto 11/20...
Procesando texto 12/20...
Procesando texto 13/20...
Procesando texto 14/20...
Procesando texto 15/20...
Procesando texto 16/20...
Procesando texto 17/20...
Procesando texto 18/20...
Procesando texto 19/20...
Procesando texto 20/20...

Todos los resultados guardados en ..\resultados\07\a1_progresion_textual_ref.json
